# OTUs


This step generates Operational Taxonomic Units (OTUs). The following steps are executed to extract OTUs.

1. Merging paired-end read sequences
2. Filtering merged sequences using quality scores
3. Dereplicating merged sequences
4. Clustering sequences
5. Chimera removal
6. Taxonomic classification

## Merging paired-end read sequences
As the first step, we will merge forward and backward read sequences into one. The resultant sequences will be used for next steps.

```{.bash}
qiime vsearch merge-pairs \
--i-demultiplexed-seqs paired-end-demux-trimmed.qza \
--o-merged-sequences demux-joined.qza \
--o-unmerged-sequences demux-unjoined.qza

```

## Filtering merged sequences using quality scores
Now we will use merged pairs and apply quality filtering.

```{.bash}
qiime quality-filter q-score \
–-i-demux demux-joined.qza \
--p-min-quality 20 \
–-o-filtered-sequences demux-joined-filtered.qza
–-o-filter-stats demux-joined-filter-stats.qza
```


## Dereplicating merged sequences
This step identify duplicate sequences and remove them. This step reduces final sequences for clustering. 

```{.bash}
qiime vsearch dereplicate-sequences \
        --i-sequences demux-joined-filtered.qza \
        --o-dereplicated-table derep-joined-filtered_table.qza \
        --o-dereplicated-sequences derep-joined-filtered_seqs.qza

```

## Clustering sequences


```{.bash}
qiime vsearch cluster-features-de-novo \
        --i-table derep-joined-filtered_table.qza \
        --i-sequences derep-joined-filtered_seqs.qza \
        --p-perc-identity .97 \
        --o-clustered-table cluster-table.qza \
        --o-clustered-sequences cluster-seqs.qza  \
        --p-threads 8
```

## Chimera removal

```{.bash}
qiime vsearch uchime-ref \
	--i-table cluster-table.qza \
	--i-sequences  cluster-seqs.qza \
	--i-reference-sequences silva-138.1-ssu-nr99-seqs-515f-806r.qza \
	--p-threads 4 \
	--o-chimeras chimeras.qza \
    --o-nonchimeras non-chimera.qza \
    --o-stats chimera-stats.qza
```

```{.bash}
qiime feature-table filter-features \
	--i-table cluster-table.qza \
    --m-metadata-file non-chimera.qza \
    --o-filtered-table table-nc.qza"
```

```{.bash}
qiime feature-table filter-seqs \
    --i-data cluster-seqs.qza \
    --i-table table-nc.qza \
    --o-filtered-data seqs-nc.qza
```

## Taxonomic classification

```{.bash}
qiime feature-classifier classify-sklearn \
    --i-classifier silva_classifier.qza \
    --i-reads seqs-nc.qza \
    --o-classification seqs-nc-otus.qza
```


```